# Mapping geographic observational data using the Google API
This is a Juypter Notebook that demonstrates using the Google API to graph data from iNaturalist

In [ ]:
import csv
import pprint

## Importing the gmaps library
The gmaps library lets us interact with the Google Maps API from within Jupyter. The documentation for using the library is online: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html

In [ ]:
import gmaps

In [ ]:
mykey = "AI...." # put your API key here

gmaps.configure(api_key=mykey) # Your Google API key

## Testing the Google Maps API with the traffic dataset demo
This is a demo using a dataset provided by google, that shows San Francisco taxi data on Google Maps using a heatmap layer.

In [ ]:
import gmaps.datasets

# get the location data, which is a set of (lat, lon) coordinates
locations = gmaps.datasets.load_dataset("taxi_rides")

# make a figure
fig = gmaps.figure()

# make a heatmap layer from the location data
taxi_heatmap = gmaps.heatmap_layer(locations)

# add the layer to the figure
fig.add_layer(taxi_heatmap)

# evaluate the figure, to display it
fig

## Working with iNaturalist data
The CSV file provided shows observation data collected at SSU

In [ ]:
with open('data/inaturalist-observations-2021.02.16.csv', newline='') as csvfile:
    observations = [r for r in csv.DictReader(csvfile)]

In [ ]:
# We can look at one of the pieces of data, to see what a single piece of data "looks like"
pprint.pprint(observations[0])

## Example: SSU observations / heatmap layer
Plots all SSU iNaturalist data as a heatmap

In [ ]:
# build a list of (lat,lon) coordinates from the observational data
all_sightings = [(float(d['latitude']), float(d['longitude'])) for d in observations]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(all_sightings)
heatmap_layer.max_intensity = 100
fig.add_layer(heatmap_layer)
fig

## Example: SSU birds / heatmap layer
Plots all SSU iNaturalist bird data as a heatmap

In [ ]:
# here is another example of how to build a list of (lat,lon) coordinated part of the observational data
bird_sightings = []
for d in observations:
    
    # get the coordinate of the sighting
    lat = float(d['latitude'])
    lon =  float(d['longitude'])
    pt = (lat, lon)

    if d['iconic_taxon_name'] == 'Aves':
        # if its a bird, remember the coordinates
        bird_sightings.append(pt)        

fig = gmaps.figure(map_type='SATELLITE') # map types are 'ROADMAP' (default), 'SATELLITE', or 'HYBRID'
heatmap_layer = gmaps.heatmap_layer(bird_sightings)
heatmap_layer.point_radius = 20
fig.add_layer(heatmap_layer)
fig

## Example: SSU birds / marker layer
Plots all SSU iNaturalist bird data as markers, where each marker has an info box

In [ ]:
# This will be the template for generating info boxes
info_box_template = """
{common_name}<br>
<img src={image_url} width=200pts>
"""

# a subset of the observational data
bird_data = [d for d in observations if d['iconic_taxon_name'] == 'Aves']

# the (lat, lon) coordinates for the bird_data
bird_sightings = [(float(d['latitude']), float(d['longitude'])) for d in bird_data]

# the info boxes generated from the bird_data
bird_info = [info_box_template.format(**bird) for bird in bird_data]

# build the map, adding the markers and info boxes
fig = gmaps.figure()
markers = gmaps.marker_layer(bird_sightings, info_box_content=bird_info)
fig.add_layer(markers)
fig

## Example: SSU birds / symbol layer
Plots all SSU iNaturalist bird data as symbols, where each symbol can be different sizes

In [ ]:
bird_data = [d for d in observations if d['iconic_taxon_name'] == 'Aves']
bird_sightings = [(float(d['latitude']), float(d['longitude'])) for d in bird_data]

fig = gmaps.figure()
points_layer = gmaps.symbol_layer(bird_sightings, fill_color='green', stroke_color='green', scale=2)
fig.add_layer(points_layer)
fig